# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [ ]:
import pandas as pd
import numpy as np
import torch

from torchtext import datasets

from torchtext.data import Field, LabelField
from torchtext.data import BucketIterator

from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm

В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader-а.

In [ ]:
TEXT = Field(sequential=True, lower=True, include_lengths=True)  # Поле текста
LABEL = LabelField(dtype=torch.float)  # Поле метки

In [ ]:
SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Датасет на котором мы будем проводить эксперементы это комментарии к фильмам из сайта IMDB.

In [ ]:
train, test = datasets.IMDB.splits(TEXT, LABEL)  # загрузим датасет
train, valid = train.split(random_state=random.seed(SEED))  # разобьем на части

In [ ]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True,
    device = device)

## RNN

Для начала попробуем использовать рекурентные нейронные сети. На семинаре вы познакомились с GRU, вы можете также попробовать LSTM. Можно использовать для классификации как hidden_state, так и output последнего токена.

In [ ]:
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout = dropout, bidirectional = bidirectional)  # YOUR CODE GOES HERE

        self.drop = nn.Dropout(p=0.2)
        
        self.fc = nn.Linear(2*hidden_dim, 1)

        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        # cell arg for LSTM, remove for GRU
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        hidden = self.drop(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        output = self.fc(hidden)
        
       
                
        #hidden = [batch size, hid dim * num directions] or [batch_size, hid dim * num directions]
            
        return output.squeeze(1) #self.fc(hidden)

Поиграйтесь с гиперпараметрами

In [ ]:
vocab_size = len(TEXT.vocab)
emb_dim = 200
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience=3
learning_rate = 0.001

In [ ]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

In [ ]:
model = model.to(device)

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.BCEWithLogitsLoss()
2
max_epochs = 25

Обучите сетку! Используйте любые вам удобные инструменты, Catalyst, PyTorch Lightning или свои велосипеды.

In [ ]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        input_embeds = batch.text[0].to(device)
        text_lenghts = batch.text[1].to(device)
        labels = batch.label.to(device)

        opt.zero_grad()
        prediction = model(input_embeds, text_lenghts)
        #print(prediction)
        loss = loss_func(prediction, labels)
        loss.backward()
        train_loss += loss
        
        #pbar.update(labels.size(0))
        opt.step()

    train_loss /= len(train_iter)

    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")

    with torch.no_grad():
      for it, batch in pbar:

        input_embeds = batch.text[0].to(device)
        text_lenghts = batch.text[1].to(device)
        labels = batch.label.to(device)
        
        prediction = model(input_embeds,text_lenghts)
        loss = loss_func(prediction, labels)

        val_loss += loss
        #valid_acc += (labels == torch.max(prediction, axis=1)[1]).float().mean()

      val_loss /= len(valid_iter)

    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

Epoch: 1, Training Loss: 0.6675910949707031, Validation Loss: 0.6656174659729004


Epoch: 2, Training Loss: 0.611029863357544, Validation Loss: 0.6177912354469299


Epoch: 3, Training Loss: 0.39481547474861145, Validation Loss: 0.4284602403640747


Epoch: 4, Training Loss: 0.18953397870063782, Validation Loss: 0.5180826783180237


Epoch: 5, Training Loss: 0.07991315424442291, Validation Loss: 0.6517079472541809


<All keys matched successfully>

Посчитайте f1-score вашего классификатора на тестовом датасете.

**Ответ**: 75.5%

In [ ]:
from sklearn.metrics import f1_score
def f1(preds, y):

    rounded_preds = torch.round(torch.sigmoid(preds)).cpu()
     
    acc = f1_score(rounded_preds.cpu(), y.cpu())
    return acc

In [ ]:
def evaluate(model):
  epoch_acc = 0
  model.eval()
    
  with torch.no_grad():
    pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
    for it, batch in pbar:
      input_embeds = batch.text[0].to(device)
      text_lenghts = batch.text[1].to(device)
      labels = batch.label.to(device)
        
      prediction = model(input_embeds,text_lenghts)

      acc = f1(prediction, batch.label)
      epoch_acc += acc.item()
  return (epoch_acc / len(test_iter))

In [ ]:
evaluate(model)

0.7556985718385166

## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что как правило сентимент содержат словосочетания из двух-трех слов, например "очент хороший фильм" или "невероятноя скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

In [ ]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first тк мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

TEXT.build_vocab(trn)
LABEL.build_vocab(trn)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort = False,
        sort_within_batch=False,
        device=device,
        repeat=False,
)

Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim))` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_size[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [ ]:
class CNN(nn.Module):
    def __init__(
        self,
        vocab_size,
        emb_dim,
        out_channels,
        kernel_sizes,
        dropout=0.5,
    ):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.conv_0 = nn.Conv1d(in_channels = emb_dim, out_channels= out_channels, kernel_size= kernel_sizes[0])  # YOUR CODE GOES HERE
        
        self.conv_1 = nn.Conv1d(in_channels = emb_dim, out_channels= out_channels, kernel_size= kernel_sizes[1])  # YOUR CODE GOES HERE
        
        self.conv_2 = nn.Conv1d(in_channels = emb_dim, out_channels= out_channels, kernel_size= kernel_sizes[2])  # YOUR CODE GOES HERE
        
        self.fc = nn.Linear(len(kernel_sizes) * out_channels, 1)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, text):

        
        embedded = self.dropout(self.embedding(text))
        
        embedded = embedded.permute(0,2,1)  # may be reshape here
        
        
        conved_0 = F.relu(self.conv_0(embedded))  # may be reshape here
        conved_1 = F.relu(self.conv_1(embedded))  # may be reshape here
        conved_2 = F.relu(self.conv_2(embedded))  # may be reshape here
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[-1]).squeeze(-1)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[-1]).squeeze(-1)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[-1]).squeeze(-1)
        
        cat = torch.cat((pooled_0, pooled_1, pooled_2), dim=-1)
            
        return self.fc(self.dropout(cat))

In [ ]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
out_channels=64
dropout = 0.5
dim = 300
patience = 3

model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=out_channels,
            kernel_sizes=kernel_sizes, dropout=dropout)

In [ ]:
model.to(device)

CNN(
  (embedding): Embedding(202904, 300)
  (conv_0): Conv1d(300, 64, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(300, 64, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(300, 64, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=0.002)
loss_func = nn.BCEWithLogitsLoss()

In [ ]:
max_epochs = 2

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for labels, text in iterator:

      input_embeds = text.to(device)
      labels = label.to(device)
      

      optimizer.zero_grad()



      train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        input_embeds = batch.text.to(device)
        labels = batch.label.to(device)
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        #acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        #epoch_acc += acc.item()
        
    return epoch_loss / len(iterator)

In [ ]:
traint(model)

In [ ]:
epoch_loss = 0
epoch_acc = 0
    
model.train()
num=0

for batch in train_iter:
  num+=1
  input_embeds = batch.text.to(device)
  labels = batch.label.to(device)
  #print(input_embeds)
  #print(labels.shape)
  #break

  opt.zero_grad()

  predictions = model(input_embeds).squeeze(1)
  #print(predictions)
  #print(predictions.shape)
  #break

  loss = loss_func(predictions, labels)
  #print(loss)

  loss.backward()
  opt.step()
  epoch_loss += loss.item()

print(epoch_loss/len(train_iter))




0.5557311022803731


In [ ]:
epoch_acc = 0
model.eval()
num=0
    
with torch.no_grad():
  pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
  for it, batch in pbar:
    input_embeds = batch.text.to(device)
      #text_lenghts = batch.text[1].to(device)
    labels = batch.label.to(device)
        
    prediction = model(input_embeds).squeeze(1)

    acc = f1(torch.sigmoid(prediction), batch.label)
    epoch_acc += acc.item()
print(epoch_acc / len(test_iter))

0.5029931972789116


In [ ]:
with torch.no_grad():
  pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
  for it, batch in pbar:
    input_embeds = batch.text.to(device)
      #text_lenghts = batch.text[1].to(device)
    labels = batch.label.to(device)
        
    prediction = model(input_embeds).squeeze(1)
    print(prediction)
    print(torch.round(torch.sigmoid(prediction)))
    break

tensor([-2.9884e+00,  5.7420e-01,  9.1486e-01, -1.0933e+00, -1.6923e+00,
        -1.6985e+00, -6.7611e-01, -1.2621e+00,  1.5106e+00, -2.1503e-01,
        -1.1490e+00, -1.4396e+00, -2.0113e+00, -1.2953e+00, -5.2229e-01,
         1.1215e+00, -1.8049e+00, -5.4961e+00, -1.0996e+00,  1.0884e+00,
         1.9180e-01, -1.4470e+00, -3.0425e-01, -1.8973e+00, -1.4387e+00,
        -1.9112e+00, -2.9289e+00, -8.1022e-01, -1.3421e+00,  7.3998e-02,
        -7.9343e-01, -8.5904e-01, -9.6226e-01, -3.0616e+00, -1.1715e+00,
        -1.3430e-01,  1.3746e+00, -1.4685e+00, -8.7472e-01, -1.4082e+00,
        -1.6336e+00, -1.1130e+00, -5.7160e-01, -2.4585e+00,  5.1753e-03,
        -6.6934e-01, -1.8869e+00, -2.3461e+00, -1.0265e+00,  1.5271e+00,
        -2.1649e+00, -1.2142e+00, -2.4826e+00, -6.3772e-01, -1.7455e+00,
        -2.0273e-01, -2.6620e+00, -4.1370e-01, -2.2515e+00, -3.6667e+00,
        -1.9707e+00, -4.0991e-01, -6.5158e-01, -7.4091e-01, -6.3170e-01,
         1.2428e+00, -3.3392e-01, -2.8194e+00,  4.9

Обучите!

In [ ]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):

    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        input_embeds = batch.text.to(device)
        #text_lenghts = batch.text[1].to(device)
        labels = batch.label.to(device)
        #print(input_embeds.shape)
        opt.zero_grad()
        prediction = model(input_embeds).squeeze(1)
        #print(prediction.shape)
        #print(prediction)
        #loss.backward()
        loss = loss_func(prediction, labels)
        loss.backward()
        opt.step()
        train_loss += loss
        #print(train_loss)
        
        #pbar.update(labels.size(0))
        
    train_loss /= len(train_iter)


    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")  
    with torch.no_grad():
       for it, batch in pbar:
         
         input_embeds = batch.text.to(device)
         labels = batch.label.to(device)
         prediction = model(input_embeds).squeeze(1)
         loss = loss_func(prediction, labels)
         val_loss += loss.item()
         #print(val_loss)
      
    val_loss /= len(val_iter)

    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

Epoch: 1, Training Loss: 0.6112194061279297, Validation Loss: 0.4605503370364507


Epoch: 2, Training Loss: 0.4749358296394348, Validation Loss: 0.41553501089413963


<All keys matched successfully>

Посчитайте f1-score вашего классификатора.

**Ответ**:79.6

In [ ]:
def evaluate(model):
  epoch_acc = 0
  model.eval()
  num=0
    
  with torch.no_grad():
    pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
    for it, batch in pbar:
      input_embeds = batch.text.to(device)
      #text_lenghts = batch.text[1].to(device)
      labels = batch.label.to(device)
        
      prediction = model(input_embeds).squeeze(1)

      acc = f1(prediction, batch.label)
      epoch_acc += acc.item()
  return (epoch_acc / len(test_iter))

In [ ]:
evaluate(model)

0.4041426211509255

In [ ]:
with torch.no_grad():
  pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
  for it, batch in pbar:
    input_embeds = batch.text.to(device)
      #text_lenghts = batch.text[1].to(device)
    labels = batch.label.to(device)
        
    prediction = model(input_embeds).squeeze(1)
    print(prediction)
    print(torch.round(prediction))
    break

tensor([-3.8823,  0.5790, -0.5096, -1.5463, -1.8269, -4.4702, -1.3326, -1.2267,
        -0.8888, -0.7686, -1.8222, -1.5772, -2.4130, -2.2701, -0.2438, -1.2837,
        -2.5302, -3.5716,  0.2233, -2.3953, -0.1153, -1.9643,  0.1647, -3.0155,
        -1.9077, -2.0267, -3.8268, -3.1861, -3.2277, -1.0492, -1.2930, -0.7340,
        -1.8680, -2.6593,  0.6928, -1.6169, -2.2522, -4.1065, -2.0566, -2.2169,
        -2.1757, -1.1752, -1.8223, -2.4413, -0.1925, -2.4383, -0.8964, -1.6174,
        -2.5001,  0.5667, -2.5757, -1.5153, -2.2805,  0.3870, -2.6773, -1.5859,
        -3.9561, -1.7396, -2.0480, -3.4019, -3.2841, -2.7642, -0.3878,  0.4486,
        -2.6544, -0.8827,  0.1480, -3.4433,  0.6049, -4.0182, -2.1597, -1.3348,
         0.6495, -2.1342, -1.8090, -0.2676, -3.1601, -0.7260, -1.6998, -2.9786,
        -2.3902, -3.5383, -0.6852, -1.3474, -2.8496, -0.8129, -4.1211, -1.5868,
        -2.0463, -0.7259, -0.3903, -2.1635, -2.6854, -0.9459, -1.6384, -0.1541,
        -1.9270, -2.3195, -2.1121, -1.13

## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [ ]:
!pip install -q captum

     |████████████████████████████████| 1.4MB 12.5MB/s 


In [ ]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

In [ ]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))


# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in TEXT.tokenize(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)

    print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [ ]:
interpret_sentence(model, 'It was a bullshit !', label=1)
interpret_sentence(model, 'Best film ever', label=0)
interpret_sentence(model, 'Such a great show!', label=0)
interpret_sentence(model, 'It was a horrible movie', label=1)
interpret_sentence(model, 'I\'ve never watched something as bad', label=1)
interpret_sentence(model, 'It is a disgusting movie!', label=1)

pred:  neg ( 0.99 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.99 ) , delta:  tensor([4.4200e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.48 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)
pred:  neg ( 1.00 ) , delta:  tensor([7.0054e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.81 ) , delta:  tensor([9.2749e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.73 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)


Попробуйте добавить свои примеры!

In [ ]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
neg,neg (0.99),neg,-0.82,It was a bullshit ! pad pad
pos,neg (0.99),neg,-1.29,Best film ever pad pad pad pad
pos,pos (0.48),neg,-1.33,Such a great show! pad pad pad
neg,neg (1.00),neg,-0.04,It was a horrible movie pad pad
neg,neg (0.81),neg,-1.30,I've never watched something as bad pad
neg,neg (0.73),neg,-1.48,It is a disgusting movie! pad pad


## Эмбэдинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [ ]:


TEXT.build_vocab(trn, vectors='glove.6B.100d')# YOUR CODE GOES HERE
# подсказка: один из импортов пока не использовался, быть может он нужен в строке выше :)
LABEL.build_vocab(trn)

word_embeddings = TEXT.vocab.vectors

kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
dropout = 0.5
dim = 100

In [ ]:
train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=True,
        sort_key= lambda x: len(x.text),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

In [ ]:
vocab_size

202904

In [ ]:
model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=64,
            kernel_sizes=kernel_sizes, dropout=dropout)

word_embeddings = TEXT.vocab.vectors
print(word_embeddings.shape)

prev_shape = model.embedding.weight.shape
print(prev_shape)
model.embedding.weight = nn.Parameter(word_embeddings)
print(model.embedding.weight.shape)
#model.embedding.weight.data.copy_(word_embeddings) 

assert prev_shape == model.embedding.weight.shape
model.to(device)

opt = torch.optim.Adam(model.parameters())

torch.Size([202904, 100])
torch.Size([202904, 100])
torch.Size([202904, 100])


Вы знаете, что делать.

In [ ]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        input_embeds = batch.text.to(device)
        #text_lenghts = batch.text[1].to(device)
        labels = batch.label.to(device)
        #print(input_embeds.shape)
        opt.zero_grad()
        prediction = model(input_embeds)
        #print(prediction)
        loss = loss_func(prediction.squeeze(1), labels)
        loss.backward()
        train_loss += loss
        
        #pbar.update(labels.size(0))
        opt.step()

    train_loss /= len(train_iter)
    val_loss = 0.0

    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    with torch.no_grad():

       for it, batch in pbar:
         input_embeds = batch.text.to(device)
         labels = batch.label.to(device)
         prediction = model(input_embeds)
         loss = loss_func(prediction.squeeze(1), labels)
         val_loss += loss

    val_loss /= len(val_iter)

    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

Epoch: 1, Training Loss: 0.21001717448234558, Validation Loss: 0.3536396324634552


Epoch: 2, Training Loss: 0.12326372414827347, Validation Loss: 0.3518514633178711


Epoch: 3, Training Loss: 0.06679879873991013, Validation Loss: 0.34208694100379944


Epoch: 4, Training Loss: 0.0350390300154686, Validation Loss: 0.36224639415740967


Epoch: 5, Training Loss: 0.0195772685110569, Validation Loss: 0.3910897374153137


<All keys matched successfully>

Посчитайте f1-score вашего классификатора.

**Ответ**:84%

In [ ]:
def evaluate(model):
  epoch_acc = 0
  model.eval()
  pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)  
  with torch.no_grad():
    
    for it, batch in pbar:
      input_embeds = batch.text.to(device)
      #text_lenghts = batch.text[1].to(device)
      labels = batch.label.to(device)
        
      prediction = model(input_embeds)
 
      acc = f1(prediction, batch.label)
      epoch_acc += acc.item()
  return (epoch_acc / len(test_iter))

In [ ]:
evaluate(model)

0.8417231561426508

Проверим насколько все хорошо!

In [ ]:
PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)
vis_data_records_ig = []

interpret_sentence(model, 'It was a fantastic performance !', label=0)
interpret_sentence(model, 'Best film ever', label=0)
interpret_sentence(model, 'Such a great show!', label=0)
interpret_sentence(model, 'It was a horrible movie', label=1)
interpret_sentence(model, 'I\'ve never watched something as bad', label=1)
interpret_sentence(model, 'It is a disgusting movie!', label=1)

pred:  pos ( 0.05 ) , delta:  tensor([4.9312e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.87 ) , delta:  tensor([5.7842e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.10 ) , delta:  tensor([0.0003], device='cuda:0', dtype=torch.float64)
pred:  neg ( 1.00 ) , delta:  tensor([1.7883e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.57 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.99 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)


In [ ]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.05),neg,-1.77,It was a fantastic performance ! pad
pos,neg (0.87),neg,-0.92,Best film ever pad pad pad pad
pos,pos (0.10),neg,-1.69,Such a great show! pad pad pad
neg,neg (1.00),neg,0.67,It was a horrible movie pad pad
neg,neg (0.57),neg,-1.71,I've never watched something as bad pad
neg,neg (0.99),neg,0.42,It is a disgusting movie! pad pad
